In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from multiprocess import Pool
from tqdm import tqdm
import requests
import urllib.request
from glob import glob
import os
import time

In [2]:
## my own directory
os.chdir("/g/data/k10/dl6968/Semi-variogram_AU/")

In [3]:
def scraper_daily(bom_id, obs_code, start_year=1800):
    # Replace with your URL
    url = f'http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_stn_num={bom_id}&p_display_type=availableYears&p_nccObsCode={obs_code}'
    
    
    # Add headers to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Check if the request was successful
        
        # Assuming the URL gives you a single line of text
        text_line = response.text.strip()
        # print(text_line)
    
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    
    pc_idx = text_line.rfind(":")
    p_c = text_line[pc_idx+1:].strip()
    
    ## combine as url
    
    bom_url = f"http://www.bom.gov.au/jsp/ncc/cdio/weatherData/av?p_display_type=dailyZippedDataFile&p_stn_num={bom_id}&p_nccObsCode={obs_code}&p_c={p_c}&p_startYear={start_year}"
    ### download zip files
    opener = urllib.request.URLopener()
    opener.addheader('User-Agent', 'whatever')
    filename, headers = opener.retrieve(bom_url, f'{out_path}{station_id}.zip')

In [4]:

# Define an empty list to store rows
data = []

# Read the file line by line
with open('./data/BoM_daily_station_list.txt', 'r') as file:
    for i, line in enumerate(file):
        # Extract columns using slicing or custom splitting
        ID = line[0:7].strip()
        Location = line[8:38].strip()
        Latitude = float(line[49:59].strip())
        Longitude = float(line[59:67].strip())
        Start_Month = line[67:71].strip()
        Start_Year = line[71:77].strip()
        End_Month = line[77:81].strip()
        End_Year = line[81:85].strip()
        Value1 = float(line[86:94].strip())
        Value2 = int(line[95:99].strip())
        Code = line[100:101].strip()
        
        # Append the parsed row to the data list
        data.append([ID, Location, Latitude, Longitude, Start_Month, Start_Year,
                     End_Month, End_Year, Value1, Value2, Code])

# Convert to DataFrame
df = pd.DataFrame(data, columns=['ID', 'Location', 'Latitude', 'Longitude', 'Start_Month', 'Start_Year',
                                 'End_Month',  'End_Year', 'Years', 'Percent', 'Code'])



In [5]:
df.to_csv("./data/BoM_daily_stations.csv")

In [6]:
## this is NCI directory
out_path = "/g/data/w40/dl6968/BoM_daily_stations/"

In [7]:
obs_code = 136
station_id = 40592
start_year = df["Start_Year"].iloc[i] 
bom_id = str(station_id).zfill(6)
scraper_daily(bom_id, obs_code, start_year)

/jobfs/127143431.gadi-pbs/ipykernel_2076892/2732621276.py:4: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  scraper_daily(bom_id, obs_code, start_year)


In [13]:
for i in tqdm(range(17364,len(df["ID"])), position=0, leave=True):
    station_id = df["ID"].iloc[i]
    start_year = df["Start_Year"].iloc[i] 
    bom_id = str(station_id).zfill(6)
    scraper_daily(bom_id, obs_code, start_year)
    
    time.sleep(np.random.random(1)[0])

  0%|          | 0/571 [00:00<?, ?it/s]/jobfs/126953221.gadi-pbs/ipykernel_600280/1535862699.py:5: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  scraper_daily(bom_id, obs_code, start_year)
100%|██████████| 571/571 [13:26<00:00,  1.41s/it]


In [14]:
other_stations = []
for i in range(0,len(df["ID"])):
    station_id = df["ID"].iloc[i]
    out_file = f'{out_path}{station_id}.zip'
    if not os.path.exists(out_file):
        other_stations.append(station_id)

In [15]:
other_stations

['6053']

In [16]:
station_id=6053
start_year=1909
bom_id = str(station_id).zfill(6)
scraper_daily(bom_id, obs_code, start_year)

/jobfs/126953221.gadi-pbs/ipykernel_600280/90614768.py:4: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  scraper_daily(bom_id, obs_code, start_year)
